In [2]:
%matplotlib inline
from IPython import display

import itertools
import math
import time
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

train_dataset = datasets.MNIST(root="./data/",
                               train=True,
                               download=True,
                               transform=transform)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>